In [ ]:
# Function to read PDF and create documents and upload document into vector store
def read_pdf(file_path):
    reader = PyPDF2.PdfReader(file_path)
    texts = []
    for page in reader.pages:
        texts.append(page.extract_text())
    return texts

# Import PDF into vector store
pdf_texts = read_pdf('data.pdf')
docs = [Document(page_content=text, metadata={}) for text in pdf_texts]
pinecone_store.add_documents(docs)

In [ ]:
# import pinecone
import requests
from langchain_core.tools import tool
from typing import Annotated
from dotenv import load_dotenv
import os
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

load_dotenv()



# Initialize Pinecone
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")


embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
pinecone_store = PineconeVectorStore(embedding=embeddings, pinecone_api_key=PINECONE_API_KEY, index_name=PINECONE_INDEX_NAME)
docsearch = pinecone_store.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=embeddings)

# Define Pinecone tool
@tool
def pinecone_search(query: Annotated[str, "The query to search in Pinecone."]):
    # index = pinecone.Index("your-index-name")
    response = docsearch.query([query], top_k=5)
    return response

# Define OpenWeather tool
@tool
def get_weather(city: Annotated[str, "The city to get weather information for."]):
    api_key = os.getenv("OPENWEATHER_API_KEY")
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = base_url + "appid=" + api_key + "&q=" + city
    response = requests.get(complete_url)
    return response.json()
